In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16

In [6]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


# Перетворення даних
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Розширення каналів з 1 до 3, щоб відповідати очікуванням VGG16
x_train = np.repeat(x_train, 3, axis=3)
x_test = np.repeat(x_test, 3, axis=3)

# Зміна розміру зображень до 32x32 пікселів
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])

# Нормалізація даних
x_train, x_test = x_train / 255.0, x_test / 255.0

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
# Завантаження базової моделі VGG16 без верхніх шарів
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Замороження ваг базової моделі
vgg16_base.trainable = False

# Додавання нових шарів до базової моделі
x = GlobalAveragePooling2D()(vgg16_base.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)


58889256/58889256 [==============================] - 0s 0us/step


In [8]:
# Створення нової моделі
vgg16_model = Model(inputs=vgg16_base.input, outputs=output)

# Компіляція моделі
vgg16_model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

# Початкове навчання моделі
vgg16_history = vgg16_model.fit(x_train, y_train, epochs=10, batch_size=256, validation_split=0.2)

# Збереження моделі та історії навчання
vgg16_model.save('vgg16_model.h5')
np.save('vgg16_history.npy', vgg16_history.history)

Epoch 1/10
188/188 [==============================] - 658s 4s/step - loss: 0.9732 - accuracy: 0.6675 - val_loss: 0.5808 - val_accuracy: 0.7958
Epoch 2/10
188/188 [==============================] - 656s 3s/step - loss: 0.6086 - accuracy: 0.7881 - val_loss: 0.5007 - val_accuracy: 0.8179
Epoch 3/10
188/188 [==============================] - 654s 3s/step - loss: 0.5325 - accuracy: 0.8131 - val_loss: 0.4750 - val_accuracy: 0.8267
Epoch 4/10
188/188 [==============================] - 640s 3s/step - loss: 0.4956 - accuracy: 0.8240 - val_loss: 0.4436 - val_accuracy: 0.8362
Epoch 5/10
188/188 [==============================] - 655s 3s/step - loss: 0.4722 - accuracy: 0.8317 - val_loss: 0.4308 - val_accuracy: 0.8421
Epoch 6/10
188/188 [==============================] - 654s 3s/step - loss: 0.4507 - accuracy: 0.8385 - val_loss: 0.4180 - val_accuracy: 0.8447
Epoch 7/10
188/188 [==============================] - 629s 3s/step - loss: 0.4383 - accuracy: 0.8433 - val_loss: 0.4063 - val_accuracy: 0.8481

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
